In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math
from pyts.image import RecurrencePlot, GramianAngularField
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model, Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from collections import deque
import warnings
from pickle import dump, load

file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [38]:
# currencies = ['Usd_Chf', 'Gbp_Usd', 'Usd_Cad', 'Aud_Usd', 'Eur_Usd', 'Usd_Jpy', 'Nzd_Usd']
currencies = ['Gbp_Usd']
years = '2016-2022'

size = 3 / 3
dfs, df_longs = [], []

for currency_pair in currencies:
    df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{years}.csv')
    df.Date = pd.to_datetime(df.Date)
    cutoff_idx = int((1 - size) * len(df))
    df = df.iloc[cutoff_idx:, :]
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    dfs.append(df)

    df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_M30_{years}.csv')
    df_long.Date = pd.to_datetime(df_long.Date)
    df_long.dropna(inplace=True)
    df_long.reset_index(drop=True, inplace=True)
    df_longs.append(df_long)

In [39]:
for df in dfs:
    df['sin_hour'] = np.sin(2 * np.pi * df['Date'].dt.hour / 24)
    # df['cos_hour'] = np.cos(2 * np.pi * df['Date'].dt.hour / 24)
    df['sin_day'] = np.sin(2 * np.pi * df['Date'].dt.day / 7)
    # df['cos_day'] = np.cos(2 * np.pi * df['Date'].dt.day / 7)

In [40]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def supertrend(barsdata, atr_len=3, mult=3):
    curr_atr = atr(barsdata['Mid_High'], barsdata['Mid_Low'], barsdata['Mid_Close'], lookback=atr_len)
    highs, lows = barsdata['Mid_High'], barsdata['Mid_Low']
    hl2 = (highs + lows) / 2
    final_upperband = upper_band = hl2 + mult * curr_atr
    final_lowerband = lower_band = hl2 - mult * curr_atr

    # initialize Supertrend column to True
    supertrend = [True] * len(df)

    close = barsdata['Mid_Close']
    
    for i in range(1, len(df.index)):
        curr, prev = i, i - 1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True

        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False

        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]

            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

    return supertrend, final_upperband, final_lowerband

def heikin_ashi(opens, highs, lows, closes):
    ha_close = list((opens + highs + lows + closes) / 4)
    ha_opens = []

    opens_list, closes_list = list(opens), list(closes)

    for i in range(len(ha_close)):
        if i == 0:
            ha_opens.append((opens_list[i] + closes_list[i]) / 2)

        else:
            ha_opens.append((ha_opens[i - 1] + ha_close[i - 1]) / 2)

    ha_highs = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'high': list(highs)}).max(axis=1))
    ha_lows = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'low': list(lows)}).min(axis=1))

    return ha_opens, ha_highs, ha_lows, ha_close

def trend_indicator(opens, highs, lows, closes, ema_period=50, smoothing_period=10):
    ha_open, ha_high, ha_low, ha_close = heikin_ashi(opens, highs, lows, closes)

    ha_o_ema = pd.Series.ewm(pd.DataFrame({'ha_open': ha_open}), span=ema_period).mean()
    ha_h_ema = pd.Series.ewm(pd.DataFrame({'ha_high': ha_high}), span=ema_period).mean()
    ha_l_ema = pd.Series.ewm(pd.DataFrame({'ha_low': ha_low}), span=ema_period).mean()
    ha_c_ema = pd.Series.ewm(pd.DataFrame({'ha_close': ha_close}), span=ema_period).mean()

    return pd.Series.ewm(ha_o_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_h_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_l_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_c_ema, span=smoothing_period).mean()

def qqe_mod(barsdata, rsi_period=6, smoothing=5, qqe_factor=5, qqe2_factor=1.61, threshold=3, mult=0.35, sma_length=50):
    wilders_period = rsi_period * 2 - 1

    curr_rsi = rsi(barsdata, periods=rsi_period)
    rsi_ema = pd.Series.ewm(curr_rsi, span=smoothing).mean()
    atr_rsi = abs(rsi_ema.shift(1) - rsi_ema)
    atr_rsi_ema = pd.Series.ewm(atr_rsi, span=wilders_period).mean()
    dar = pd.Series.ewm(atr_rsi_ema, span=wilders_period).mean() * qqe_factor

    newshortband = rsi_ema + dar
    newlongband = rsi_ema - dar

    rsi_ema_list = list(rsi_ema)

    longband = [0]
    for i in range(1, len(rsi_ema_list)):
        if rsi_ema_list[i - 1] > longband[i - 1] and rsi_ema_list[i] > longband[i - 1]:
            longband.append(max(longband[i - 1],newlongband[i]))

        else:
            longband.append(newlongband[i])

    shortband = [0]
    for i in range(1,len(rsi_ema_list)):
        if rsi_ema_list[i - 1] < shortband[i - 1] and rsi_ema_list[i] < shortband[i - 1]:
            shortband.append(min(shortband[i - 1],newshortband[i]))
            
        else:
            shortband.append(newshortband[i])

    longband = pd.Series(longband)
    shortband = pd.Series(shortband)

    trend = np.where(rsi_ema > longband.shift(1), 1, -1)    
    fastatrrsitl = pd.Series(np.where(trend == 1, longband, shortband))

    basis = (fastatrrsitl - 50).rolling(window=sma_length).mean()
    dev = (fastatrrsitl - 50).rolling(window=sma_length).std() * mult
    upper = basis + dev
    lower = basis - dev

    greenbar1 = rsi_ema - 50 > threshold
    greenbar2 = rsi_ema - 50 > upper
    redbar1 = rsi_ema - 50 < threshold
    redbar2 = rsi_ema - 50 < lower

    # uptrend = np.where((greenbar1 & greenbar2), True, False)
    # downtrend = np.where((redbar1 & redbar2), True, False)

    uptrend = np.where((greenbar2), True, False)
    downtrend = np.where((redbar2), True, False)

    return uptrend, downtrend

def williams_r(highs, lows, closes, length=21, ema_length=15):
    highest_highs = highs.rolling(window=length).max()
    lowest_lows = lows.rolling(window=length).min()

    willy = 100 * (closes - highest_highs) / (highest_highs - lowest_lows)
    willy_ema = pd.Series.ewm(willy, span=ema_length).mean()

    return willy, willy_ema

def squeeze(barsdata, length=20, length_kc=20, mult=1.5):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    upper_bb = m_avg + mult * m_std
    lower_bb = m_avg - mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    upper_kc = m_avg + range_ma * mult
    lower_kc = m_avg - range_ma * mult

    # Squeeze
    squeeze_on = (lower_bb > lower_kc) & (upper_bb < upper_kc)

    return squeeze_on

In [41]:
bid_ask_mid_prices_list = []

for df in dfs:
    # Add technical indicators (for additional features)
    df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
    df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
    # df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()

    df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
    df['atr_sma'] = df['atr'].rolling(window=20).mean()
    df['rsi'] = rsi(df['Mid_Close'])
    df['rsi_sma'] = df['rsi'].rolling(50).mean()
    df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
    df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
    df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
    df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])

    df['vo'] = vo(df['Volume'])

    df['willy'], df['willy_ema'] = williams_r(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

    df['squeeze_on'] = squeeze(df)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Extract the bid and ask prices and fractals and remove them from the df
    bid_ask_mid_prices = df[['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
    df.drop(['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close', 'Volume'], axis=1, inplace=True)
    bid_ask_mid_prices_list.append(bid_ask_mid_prices)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

for df_long in df_longs:
    df_long['ema200'] = pd.Series.ewm(df_long['Mid_Close'], span=200).mean()
    df_long['ema100'] = pd.Series.ewm(df_long['Mid_Close'], span=100).mean()
    # df_long['ema50'] = pd.Series.ewm(df_long['Mid_Close'], span=50).mean()

    df_long['atr'] = atr(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
    df_long['atr_sma'] = df_long['atr'].rolling(window=20).mean()
    df_long['rsi'] = rsi(df_long['Mid_Close'])
    df_long['rsi_sma'] = df_long['rsi'].rolling(50).mean()
    df_long['adx'] = adx(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
    df_long['macd'] = pd.Series.ewm(df_long['Mid_Close'], span=12).mean() - pd.Series.ewm(df_long['Mid_Close'], span=26).mean()
    df_long['macdsignal'] = pd.Series.ewm(df_long['macd'], span=9).mean()
    df_long['slowk_rsi'], df_long['slowd_rsi'] = stoch_rsi(df_long['rsi'])

    df_long['vo'] = vo(df_long['Volume'])

    df_long['willy'], df_long['willy_ema'] = williams_r(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])

    df_long['squeeze_on'] = squeeze(df_long)

    df_long.dropna(inplace=True)
    df_long.reset_index(drop=True, inplace=True)

    df_long.drop(['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close', 'Volume'], axis=1, inplace=True)

    df_long.dropna(inplace=True)
    df_long.reset_index(drop=True, inplace=True)

In [42]:
look_back_size = 50

In [7]:
buys_list = []
sells_list = []
nones_list = []

value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
spread_cutoff = 0.10
risk_reward_ratio = 1.5

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

for idx in range(len(currencies)):
    currency_pair, df, df_long, bid_ask_mid_prices = currencies[idx], dfs[idx], df_longs[idx], bid_ask_mid_prices_list[idx]
    rounding = 3 if 'Jpy' in currency_pair else 5
    buys, sells, nones = [], [], []
    trade, prev_year = None, None

    print(f'RUNNING SIMULATION FOR {currency_pair}...')

    for i in range(look_back_size, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        if prev_year is None or curr_date.year > prev_year:
            prev_year = curr_date.year
            print(prev_year)
            print(f'Buys: {len(buys)}')
            print(f'Sells: {len(sells)}')
            print(f'Nones: {len(nones)}\n')
        curr_long = df_long.loc[df_long['Date'] < curr_date]
        if len(curr_long) < look_back_size + 1:
            continue
        ema200_2, ema100_2, atr2, rsi2, rsi_sma2, vo2, macd2, macdsignal2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'atr', 'rsi', 'rsi_sma', 'vo', 'macd', 'macdsignal']]
        ema200_1, ema100_1, atr1, rsi1, rsi_sma1, vo1, adx1, atr_sma1, macd1, macdsignal1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'atr', 'rsi', 'rsi_sma', 'vo', 'adx', 'atr_sma', 'macd', 'macdsignal']]
        mid_open2, mid_close2, mid_low2, mid_high2 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 2], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        mid_open1, mid_close1, mid_low1, mid_high1 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 1], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i], ['Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High']]
        spread = abs(curr_ao - curr_bo)

        emas_buy_signal = ema200_1 < ema100_1
        emas_sell_signal = ema200_1 > ema100_1

        if trade is None:
            if emas_buy_signal:
                lowest_low = min(list(bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 12:i], 'Mid_Low']))
                
                open_price = float(curr_ao)
                pullback = float(lowest_low) - spread

                stop_loss = round(pullback, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), rounding)

                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

            elif emas_sell_signal:
                highest_high = max(list(bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 12:i], 'Mid_High']))

                open_price = float(curr_bo)
                pullback = float(highest_high) + spread

                stop_loss = round(pullback, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), rounding)

                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

        if trade is not None:
            for j in range(i, len(df)):
                curr_date = df.loc[df.index[j], 'Date']
                curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close']]

                if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
                    nones.append(trade['start_date']) 

                    trade = None
                    break


                if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
                    trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
                    day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                    if trade_amount + day_fees > 0:
                        buys.append(trade['start_date'])

                    trade = None
                    break

                if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
                    nones.append(trade['start_date'])

                    trade = None
                    break

                if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
                    trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
                    day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                    if trade_amount + day_fees > 0:
                        sells.append(trade['start_date'])

                    trade = None
                    break
    
    buys_list.append(buys)
    sells_list.append(sells)
    nones_list.append(nones)

    print(f'Buys: {len(buys)}')
    print(f'Sells: {len(sells)}')
    print(f'Nones: {len(nones)}\n')

RUNNING SIMULATION FOR Gbp_Usd...
2016
Buys: 0
Sells: 0
Nones: 0

2017
Buys: 2981
Sells: 3200
Nones: 10271

2018
Buys: 5310
Sells: 5272
Nones: 18428

2019
Buys: 7778
Sells: 8086
Nones: 27251

2020
Buys: 10011
Sells: 10655
Nones: 35047

2021
Buys: 13907
Sells: 14032
Nones: 46454

Buys: 16337
Sells: 16696
Nones: 55650



In [8]:
buy_indices_list = []
sell_indices_list = []
nones_indices_list = []
size = 1 / 6

for i in range(len(dfs)):
    buys, sells, nones, df = buys_list[i], sells_list[i], nones_list[i], dfs[i]
    
    buy_indices = [df.index[df['Date'] == curr_date] - 1 for curr_date in buys]
    sell_indices = [df.index[df['Date'] == curr_date] - 1 for curr_date in sells]
    nones_indices = [df.index[df['Date'] == curr_date] - 1 for curr_date in nones]

    np.random.shuffle(buy_indices)
    np.random.shuffle(sell_indices)
    np.random.shuffle(nones_indices)

    lower = min(len(buy_indices), len(sell_indices), len(nones_indices))
    lower = int(lower * size)

    buy_indices = buy_indices[:lower]
    sell_indices = sell_indices[:lower]
    nones_indices = nones_indices[:lower]

    buy_indices_list.append(buy_indices)
    sell_indices_list.append(sell_indices)
    nones_indices_list.append(nones_indices)

    print(len(buy_indices))
    print(len(sell_indices))
    print(len(nones_indices))

2722
2722
2722


In [44]:
for df in dfs:
    df.drop('Date', axis=1, inplace=True)
    df['squeeze_on'] = df['squeeze_on'].astype(float)

In [71]:
scalers, pcas = {}, {}
explained_variance = 0.99

for i in range(len(dfs)):
    currency_pair, df = currencies[i], dfs[i]

    scaler = StandardScaler()

    dfs[i] = scaler.fit_transform(df)

    scalers[currency_pair] = scaler

In [72]:
correct_shape = (look_back_size, dfs[0].shape[1])
correct_shape

(50, 17)

In [75]:
def get_sequential_data():
    no_actions = []
    buys = []
    sells = []

    for z in range(len(dfs)):
        print(currencies[z])
        df = dfs[z]

        buy_indices, sell_indices, nones_indices = buy_indices_list[z], sell_indices_list[z], nones_indices_list[z]

        for i in buy_indices:
            if len(i) == 1:
                i = i[0]
                seq = df[i - look_back_size + 1:i + 1, :]
                assert seq.shape == correct_shape
                buys.append([seq, np.array([0, 1, 0])])

        for i in sell_indices:
            if len(i) == 1:
                i = i[0]
                seq = df[i - look_back_size + 1:i + 1, :]
                assert seq.shape == correct_shape
                sells.append([seq, np.array([0, 0, 1])])

        for i in nones_indices:
            if len(i) == 1:
                i = i[0]
                seq = df[i - look_back_size + 1:i + 1, :]
                assert seq.shape == correct_shape
                no_actions.append([seq, np.array([1, 0, 0])])

    np.random.shuffle(no_actions)
    np.random.shuffle(buys)
    np.random.shuffle(sells)

    sequential_data = no_actions + buys + sells
    np.random.shuffle(sequential_data)

    return sequential_data

In [76]:
sequential_data = get_sequential_data()

Gbp_Usd


In [77]:
len(sequential_data)

8166

In [78]:
training_proportion = 0.70
train_test_cutoff_index = int(len(sequential_data) * training_proportion)

train_set = sequential_data[0:train_test_cutoff_index]
test_set = sequential_data[train_test_cutoff_index:]

print('Dataset shapes:')
print(len(train_set))
print(len(test_set))

Dataset shapes:
5716
2450


In [79]:
x_train = []
y_train = []

for seq, target in train_set:
  x_train.append(seq)
  y_train.append(target)

x_test = []
y_test = []

for seq, target in test_set:
  x_test.append(seq)
  y_test.append(target)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

x_train, x_test = x_train.astype(float), x_test.astype(float)

In [80]:
x_train.shape

(5716, 50, 17)

In [81]:
y_train.shape

(5716, 3)

In [82]:
# # Number of possible actions to take - determines the output dimension of the
# #  neural network
# n_actions = 3
# input_data_shape = x_train.shape[1:]

# model = Sequential()

# model.add(Conv2D(filters = 32, kernel_size = (3,3), padding ='Same', activation ='relu', input_shape = input_data_shape))
# model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))

# model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(128, activation = "relu"))
# model.add(Dropout(0.5))
# model.add(Dense(n_actions, activation = "softmax"))

n_actions = 3
input_data_shape = x_train.shape[1:]

model = Sequential()

model.add(GRU(256, return_sequences=True, input_shape=input_data_shape))
# model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(GRU(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(GRU(256, return_sequences=True))
# model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(GRU(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(GRU(256))
# model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(n_actions, activation='softmax'))

In [83]:
# Hyperparameters
n_epochs = 300
batch_size = 32
n_steps = len(x_train) // batch_size 
mean_loss = tf.keras.metrics.Mean() 
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.categorical_crossentropy
metrics = [tf.keras.metrics.CategoricalAccuracy()]
# loss_fn = tf.keras.losses.MeanAbsoluteError
# metrics = [tf.keras.metrics.MeanAbsoluteError()]

In [84]:
early_stop = EarlyStopping(monitor='val_accuracy', verbose=1, patience=n_epochs)
model_checkpoint = ModelCheckpoint('/Users/mymac/test_rnn', monitor='val_accuracy', save_best_only=True, verbose=1)

optimizer = Adam()

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [85]:
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=n_epochs,
    validation_data=(x_test, y_test),
    callbacks=[early_stop, model_checkpoint]
)

Epoch 1/300
179/179 [==============================] - ETA: 0s - loss: 0.9987 - accuracy: 0.4983
Epoch 00001: val_accuracy improved from -inf to 0.59878, saving model to /Users/mymac/test_rnn


INFO:tensorflow:Assets written to: /Users/mymac/test_rnn/assets


INFO:tensorflow:Assets written to: /Users/mymac/test_rnn/assets


179/179 [==============================] - 30s 146ms/step - loss: 0.9987 - accuracy: 0.4983 - val_loss: 0.8589 - val_accuracy: 0.5988
Epoch 2/300
179/179 [==============================] - ETA: 0s - loss: 0.8395 - accuracy: 0.5765
Epoch 00002: val_accuracy improved from 0.59878 to 0.63306, saving model to /Users/mymac/test_rnn


INFO:tensorflow:Assets written to: /Users/mymac/test_rnn/assets


...

179/179 [==============================] - 25s 141ms/step - loss: 0.7465 - accuracy: 0.6244 - val_loss: 0.7177 - val_accuracy: 0.6518
Epoch 41/300
179/179 [==============================] - ETA: 0s - loss: 0.7262 - accuracy: 0.6309
Epoch 00041: val_accuracy did not improve from 0.65184
179/179 [==============================] - 19s 107ms/step - loss: 0.7262 - accuracy: 0.6309 - val_loss: 0.7192 - val_accuracy: 0.6420
Epoch 42/300
179/179 [==============================] - ETA: 0s - loss: 0.7317 - accuracy: 0.6261
Epoch 00042: val_accuracy did not improve from 0.65184
179/179 [==============================] - 19s 108ms/step - loss: 0.7317 - accuracy: 0.6261 - val_loss: 0.7203 - val_accuracy: 0.6469
Epoch 43/300
179/179 [==============================] - ETA: 0s - loss: 0.7294 - accuracy: 0.6296
Epoch 00043: val_accuracy did not improve from 0.65184
179/179 [==============================] - 20s 110ms/step - loss: 0.7294 - accuracy: 0.6296 - val_loss: 0.7405 - val_accuracy: 0.6404
Epoch

INFO:tensorflow:Assets written to: /Users/mymac/test_rnn/assets


INFO:tensorflow:Assets written to: /Users/mymac/test_rnn/assets


179/179 [==============================] - 25s 140ms/step - loss: 0.7415 - accuracy: 0.6277 - val_loss: 0.7137 - val_accuracy: 0.6555
Epoch 46/300
179/179 [==============================] - ETA: 0s - loss: 0.7310 - accuracy: 0.6286
Epoch 00046: val_accuracy did not improve from 0.65551
179/179 [==============================] - 20s 110ms/step - loss: 0.7310 - accuracy: 0.6286 - val_loss: 0.7100 - val_accuracy: 0.6506
Epoch 47/300
179/179 [==============================] - ETA: 0s - loss: 0.7350 - accuracy: 0.6268
Epoch 00047: val_accuracy did not improve from 0.65551
179/179 [==============================] - 20s 111ms/step - loss: 0.7350 - accuracy: 0.6268 - val_loss: 0.7219 - val_accuracy: 0.6461
Epoch 48/300
179/179 [==============================] - ETA: 0s - loss: 0.7286 - accuracy: 0.6284
Epoch 00048: val_accuracy did not improve from 0.65551
179/179 [==============================] - 20s 110ms/step - loss: 0.7286 - accuracy: 0.6284 - val_loss: 0.7088 - val_accuracy: 0.6518
Epoch

KeyboardInterrupt: 

In [86]:
model = load_model('/Users/mymac/test_rnn')

In [87]:
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df['sin_hour'] = np.sin(2 * np.pi * df['Date'].dt.hour / 24)
df['sin_day'] = np.sin(2 * np.pi * df['Date'].dt.day / 7)

# Add technical indicators (for additional features)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
# df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()

df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['atr_sma'] = df['atr'].rolling(window=20).mean()
df['rsi'] = rsi(df['Mid_Close'])
df['rsi_sma'] = df['rsi'].rolling(50).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])

df['vo'] = vo(df['Volume'])

df['willy'], df['willy_ema'] = williams_r(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df['squeeze_on'] = squeeze(df)

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Extract the bid and ask prices and fractals and remove them from the df
bid_ask_mid_prices = df[['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
df.drop(['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close', 'Volume'], axis=1, inplace=True)

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [88]:
from copy import deepcopy

currency_pair = 'Gbp_Usd'

df_copy = deepcopy(df)

df_copy.drop('Date', axis=1, inplace=True)

df_copy = scalers[currency_pair].transform(df_copy)
# df_copy = pcas[currency_pair].transform(df_copy)

In [89]:
df_copy.shape

(74602, 17)

In [90]:
assert len(df) == len(df_copy)

In [100]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
spread_cutoff = 0.10
risk_reward_ratio = 1.5
proba_threshold = 0.65
reward = 0
n_wins, n_losses, n_buys, n_sells = 0, 0, 0, 0
probas = []

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

rounding = 3 if 'Jpy' in currency_pair else 5
buys, sells, nones = [], [], []
trade = None

for i in range(look_back_size, len(df)):
    curr_date = df.loc[df.index[i], 'Date']

    ema200_2, ema100_2, atr2, rsi2, rsi_sma2, vo2, macd2, macdsignal2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'atr', 'rsi', 'rsi_sma', 'vo', 'macd', 'macdsignal']]
    ema200_1, ema100_1, atr1, rsi1, rsi_sma1, vo1, adx1, atr_sma1, macd1, macdsignal1, squeeze_on = df.loc[df.index[i - 1], ['ema200', 'ema100', 'atr', 'rsi', 'rsi_sma', 'vo', 'adx', 'atr_sma', 'macd', 'macdsignal', 'squeeze_on']]
    mid_open2, mid_close2, mid_low2, mid_high2 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 2], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
    mid_open1, mid_close1, mid_low1, mid_high1 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 1], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
    curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i], ['Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High']]
    spread = abs(curr_ao - curr_bo)

    emas_buy_signal = ema200_1 < ema100_1
    emas_sell_signal = ema200_1 > ema100_1

    # sideways = adx1 < 20 or atr1 < atr_sma1
    sideways = False

    rsi_buy_signal = rsi1 > rsi_sma1
    rsi_sell_signal = rsi1 < rsi_sma1

    macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]

    macd_buy_signal = macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) == 0
    macd_sell_signal = macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) == 0

    if trade is None:
        if macd_buy_signal and emas_buy_signal and rsi_buy_signal and not squeeze_on:
            lowest_low = min(list(bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 12:i], 'Mid_Low']))
            
            open_price = float(curr_ao)
            pullback = float(lowest_low) - spread

            stop_loss = round(pullback, rounding)

            if stop_loss < open_price:
                curr_pips_to_risk = open_price - stop_loss

                if spread <= curr_pips_to_risk * spread_cutoff:
                    curr_seq = df_copy[i - look_back_size:i, :]
                    pred = model.predict(curr_seq.reshape(1, curr_seq.shape[0], curr_seq.shape[1]))
                    argmax = np.argmax(pred)
                    proba = pred[0][argmax]
                    nn_buy_signal = argmax == 1 and proba >= proba_threshold

                    if argmax == 1:
                        probas.append(proba)

                    if nn_buy_signal:
                        stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), rounding)

                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

        elif macd_sell_signal and emas_sell_signal and rsi_sell_signal and not squeeze_on:
            highest_high = max(list(bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 12:i], 'Mid_High']))

            open_price = float(curr_bo)
            pullback = float(highest_high) + spread

            stop_loss = round(pullback, rounding)

            if stop_loss > open_price:
                curr_pips_to_risk = stop_loss - open_price

                if spread <= curr_pips_to_risk * spread_cutoff:
                    curr_seq = df_copy[i - look_back_size:i, :]
                    pred = model.predict(curr_seq.reshape(1, curr_seq.shape[0], curr_seq.shape[1]))
                    argmax = np.argmax(pred)
                    proba = pred[0][argmax]
                    nn_sell_signal = argmax == 2 and proba >= proba_threshold

                    if argmax == 2:
                        probas.append(proba)

                    if nn_sell_signal:
                        stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), rounding)

                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

    if trade is not None:
        curr_date = df.loc[df.index[i], 'Date']
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close']]

        if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])
            reward += -50.0
            reward += day_fees

            n_losses += 1

            trade = None
            continue


        if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            reward += trade_amount + day_fees

            if trade_amount + day_fees > 0:
                n_wins += 1

            trade = None
            continue

        if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])
            reward += -50.0
            reward += day_fees

            n_losses += 1

            trade = None
            continue

        if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            reward += trade_amount + day_fees

            if trade_amount + day_fees > 0:
                n_wins += 1

            trade = None
            continue

In [101]:
print(f'Reward = {reward}')
print(f'Wins = {n_wins}')
print(f'Losses = {n_losses}')
print(f'Buys = {n_buys}')
print(f'Sells = {n_sells}')
print(max(probas))
print(min(probas))
print(np.array(probas).mean())

Reward = -4.044059999998339
Wins = 2
Losses = 3
Buys = 0
Sells = 5
0.7300598
0.3543726
0.61034626
